### Práctica de Formatos de datos: XML

El objetivo de esta práctica es crear una aplicación que gestionará información sobre las bibliotecas públicas y bibliobuses situados en la ciudad de Madrid, de titularidad nacional, comunitaria o local. 

Para ello se va a usar el catálogo "Bibliotecas y bibliobuses en la ciudad de Madrid" del portal de datos abiertos del Ayuntamiento de Madrid. Este catalogo ofrece datos de dirección, horario, servicios y coordenadas para su georeferenciación. La información se encuentra en:
https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=ed35401429b83410VgnVCM1000000b205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default

Y el conjunto de datos XML que hay que procesar está situado en:
https://datos.madrid.es/egob/catalogo/201747-0-bibliobuses-bibliotecas.xml

El siguiente trozo de código permite recuperar el archivo xml en cuestion, y lo almacena en disco.

In [ ]:
import urllib.request
x = urllib.request.urlopen('https://datos.madrid.es/egob/catalogo/201747-0-bibliobuses-bibliotecas.xml')
a=x.read()
a=a.decode("utf-8")
prueba=open("catalogo.xml","w")
prueba.write(str(a))
prueba.close()

__Ejercicio 1 [5 puntos]__

Se pide mostrar al usuario un listado de los nombres de las bibliotecas que contiene el archivo con el objetivo de que elija una de las bibliotecas y se le muestre de forma amigable(sin etiquetas) por pantalla la información asociada al parque que se encuentra en el fichero recuperado:

* Nombre de la biblioteca: atributo NOMBRE
* Horario: atributo HORARIO
* Equipamiento: atributo EQUIPAMIENTO
* Transporte: atributo TRANSPORTE
* Localización: atributos CLASE-VIAL,NOMBRE-VIA,NUM,LOCALIDAD,PROVINCIA,CODIGO-POSTAL 
* Teléfono: atributo TELÉFONO
* Email: atributo EMAIL  

Observar que algunos atributos pueden no aparecer en las bibliotecas. En estos casos, simplemente se mostrará vacío o con la palabra "Desconocido"

Por ejemplo si el usuario hubiera elegido la biblioteca "Biblioteca Pública Antonio Mingote (Latina)" debería mostrarse por pantalla:

__Nombre de la biblioteca:__

Biblioteca Pública Antonio Mingote (Latina)
    
__Horario:__

De lunes a viernes de 9 a 21 horas. Sala Infantil de 15 a 21 horas. Sábados de 9 a 14 horas. Los servicios de préstamo y acceso a internet finalizan 15 minutos antes del cierre de la Biblioteca.Días de cierre: Sábados: 11 de abril; 16 de mayo; todos los sábados de julio, agosto y septiembre; 26 de diciembre. Domingos: todos. Festivos:1 y 6 de enero; 9 y 10 de abril; 1, 2 y 15 de mayo; 15 de agosto; 12 de octubre; 2 y 9 de noviembre; 7, 8, 24, 25 y 31 de diciembre.

__Equipamiento:__

184 puestos de lectura y 25 para el uso de InternetWifi gratuito
    
__Transporte:__ 

Bus: 17, 138 Cercanías Renfe: Fanjul (línea C5)

__Localización:__ 

CALLE RAFAEL FINAT, 51.MADRID. MADRID 28044

__Teléfono:__ 

915 093 625

__Email:__ 

bib.latina@madrid.org

__Ejercicio 2 [5 puntos]__

Este ejercicio se pide crear un buscador sobre la información recuperada, de forma que el usuario podrá buscar bibliotecas por diferentes criterios:

 * Accesibilidad
 
 * Nombre
 
 * Barrio
 
 * Distrito
 
Como resultado debería mostrarse un listado de todas las bibliotecas que cumplen las condiciones impuestas por el usuario. Se mostrará de cada biblioteca la misma información que en el caso anterior.  En caso de no existir una biblioteca con las condiciones dada, se mostrará un mensaje informativo.

Observar que habrá que preguntar al usuario por cada uno de los criterios, y éste tendrá que seleccionar un valor por cada criterio o no seleccionarlo. Después de mostrarle la información, se le volverá a preguntar si quiere seguir buscando. Entre las opciones del menú deberá existir una que sea para finalizar la búsqueda.

Normas de entrega

   * Fecha tope de entrega: 29/10/2018
   * La entrega se realizará subiendo al campus virtual un notebook de Jupyter con la solución. El archivo tendrá como nombre FormatosII_GrupoX donde X será el número de grupo correspondiente.
